In [1]:
import numpy as np
import pandas as pd

# Setting up and running a scikit-learn pipeline

A typical ML workflow is as follows:

1. split data into training/testing sets.
2. each of them goes through data cleaning/preprocessing.
3. they are then used as input for a ML model.

To avoid repetitive coding (first dealing with the training set, then with the testing set), it is possible to write a scikit-learn **pipeline**.\
A pipeline links every step of the data analysis, where  the output of a given step is used as the input for the next step.

The syntax of a pipeline is as follows:\
Pipeline(steps = [(‘step name’, transform function), …])

Pipeline writing takes advantage of the **ColumnTransformer** feature in scikit-learn.\
ColumnTransformer transforms groups of dataframe columns independently, then combines them at a later stage. This is particularly useful for data preprocessing. 

Additional advantages:
- Less prone to (copy/paste) mistakes.
- Workflow easier to understand.
- Less prone to data leakage

## 1. The dataset

### 1.1 Dataset presentation

A given company hires data scientists if they follow lectures and successfully pass some tests designed by the company.\
The company wants to know which candidates are really interested in working for the company, or if successful candidates will be looking for a new employment.\
(It takes time and costs money to train candidates, which are lost if the candidates are not staying.\
The available information includes demographics, education, experience of the candidates, either as numerical or as categorical variables.

In [2]:
path_data = '../data/Kaggle_Job_Change_of_Data_Scientists/'
data = pd.read_csv(path_data+'aug_train.csv', engine='python')

In [3]:
data.columns

Index(['enrollee_id', 'city', 'city_development_index', 'gender',
       'relevent_experience', 'enrolled_university', 'education_level',
       'major_discipline', 'experience', 'company_size', 'company_type',
       'last_new_job', 'training_hours', 'target'],
      dtype='object')

In [4]:
data.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0
3,33241,city_115,0.789,NaN,No relevent experience,NaN,Graduate,Business Degree,<1,NaN,Pvt Ltd,never,52,1.0
4,666,city_162,0.767,Male,Has relevent experience,no_enrollment,Masters,STEM,>20,50-99,Funded Startup,4,8,0.0


### 1.2 Dataset preprocessing

Some useful features:
- are formatted as string
- because they contain information such as '<1' or '>20'
- or although they contain Boolean information (Has/Has no) regarding experience.
They need to be preprocessed before they can be used by the model

In [5]:
# Prepare dictionaries of ordinal features
# Note: relevant written 'relevent' in the input file' 

relevant_experience_map = {
    'Has relevent experience':  1,
    'No relevent experience':   0}

experience_map = {
    '<1'      :    0,
    '1'       :    1,
    '2'       :    2,
    '3'       :    3,
    '4'       :    4,
    '5'       :    5,
    '6'       :    6,
    '7'       :    7,
    '8'       :    8,
    '9'       :    9,
    '10'      :    10,
    '11'      :    11,
    '12'      :    12,
    '13'      :    13,
    '14'      :    14,
    '15'      :    15,
    '16'      :    16,
    '17'      :    17,
    '18'      :    18,
    '19'      :    19,
    '20'      :    20,
    '>20'     :    21} 
    
last_new_job_map = {
    'never'        :    0,
    '1'            :    1,
    '2'            :    2,
    '3'            :    3,
    '4'            :    4,
    '>4'           :    5}

In [6]:
# Transform categorical features into numerical features by mapping the previous dictionaries

def preformat(df_init):
    df = df_init.copy()
    df.loc[:,'relevent_experience'] = df['relevent_experience'].map(relevant_experience_map)
    df.loc[:,'last_new_job'] = df['last_new_job'].map(last_new_job_map)
    df.loc[:,'experience'] = df['experience'].map(experience_map)

    return df

In [7]:
data2 = preformat(data)

## 2. First level of the pipeline: Encoding

Columns of different nature (numerical/categorical will be encoded differently)

In [8]:
num_cols = ['city_development_index','relevent_experience', 'experience','last_new_job', 'training_hours']
cat_cols = ['gender', 'enrolled_university', 'education_level', 'major_discipline', 'company_size', 'company_type']

### 2.1 Encoding numerical features

Numerical features:
1. SimpleImputer to fill in missing values with the mean of the column.
2. MinMaxScaler to scale the values to ranges from 0 to 1 (in order to improve the model performance).

In [9]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

num_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('scale',MinMaxScaler())
])

### 2.2 Encoding categorical features

Categorical features:

1. SimpleImputer to fill in the missing values with the most frequent value in the column.
2. OneHotEncoder to emcode categorical features.

In [10]:
# SimpleImputer already imported
from sklearn.preprocessing import OneHotEncoder
# Pipeline  already imported

cat_pipeline = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='most_frequent')),
    ('one-hot',OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

IMPORTANT NOTE: (handle_unknown=’ignore’ is specified to prevent errors with unseen categories in the testing set\
(categories present in the testing set but not present in the training set).

### 2.3 Using ColumnTransfomer to group the two branches of the pipeline

he syntax of a ColumnTransformer is as follows:\

ColumnTransformer(transformers=[(‘step name’, transform function,cols), …])

In [11]:
from sklearn.compose import ColumnTransformer

col_trans = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

Note 1: remainder=’drop’ is specified to ignore the other dataframe columns.\
Note 2: n_job = -1 to use all processors in parallel.

## 3. Second level of the pipeline: Modelling

We use a logistic regression algorithm to classify the candidates 

In [12]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(random_state=0)
first_pipeline = Pipeline(steps=[
    ('col_trans', col_trans),
    ('model', classifier)
])

## 4. Display the pipeline

It is possible to display the pipeline.\
Clicking on the image provides the details of each step. 

In [13]:
from sklearn import set_config

set_config(display='diagram')
display(first_pipeline)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

Reminder: Alternatively, it is possible to use the utility function make_pipeline for constructing pipelines:
The names are filled in automatically:

In [63]:
from sklearn.svm import SVC
from sklearn.decomposition import PCA

from sklearn.pipeline import make_pipeline
alt_pipeline = make_pipeline(PCA(), SVC())
display(alt_pipeline)

Pipeline(steps=[('pca', PCA()), ('svc', SVC())])

## 5. Run the pipeline

### 5.1 Train/test split

Note 1: We could use the testing data provided together with the training data we used as input.\
To better match a true use case, we pretend that the input data in the full dataset, and we perform again a train/test split.\
Note 2: Specifying stratify=y ensures that the relative frequency of the categories is approximately preserved for both the training and the testing set.

In [14]:
from sklearn.model_selection import train_test_split

X = data2[num_cols+cat_cols]
y = data2['target']
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

### 5.2 Fit/Predict/Score

In [15]:
# Pipeline_fit is the command to process data through the pipeline, including model fitting.
first_pipeline.fit(X_train, y_train)

Pipeline(steps=[('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

In [16]:
# Pipeline.predict is the command to predict the outcome on unseen data thanks to the model trained just above.
y_pred = first_pipeline.predict(X_test)

In [17]:
# Pipeline.score computes the 'score' of the pipeline (preprocessing + model).
# It is possible (and desirable) to test various combinations of preprocessing methods + models to achieve the best possible score 
# Here the score is the accuracy of logistic regression.
score = first_pipeline.score(X_test, y_test)
print('\nModel score:', score)


Model score: 0.7713987473903967


### 5.3 Further improving the pipeline

Although not strictly necessary, one may like to include the preformating stage in the pipeline.\
This can be done because our preformating does not include operations that could lead to data leakage, like taking the mean of a given column.\
This can be achieved by using 'FunctionTransformer' from scikit-learn

In [18]:
from sklearn.preprocessing import FunctionTransformer
preformater = FunctionTransformer(preformat)
test = preformater.transform(data)

We quickly check that preformater provides the same output as the original 'preformat' function.

In [19]:
test.head()

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,1,no_enrollment,Graduate,STEM,21.0,NaN,NaN,1.0,36,1.0
1,29725,city_40,0.776,Male,0,no_enrollment,Graduate,STEM,15.0,50-99,Pvt Ltd,5.0,47,0.0
2,11561,city_21,0.624,NaN,0,Full time course,Graduate,STEM,5.0,NaN,NaN,0.0,83,0.0
3,33241,city_115,0.789,NaN,0,NaN,Graduate,Business Degree,0.0,NaN,Pvt Ltd,0.0,52,1.0
4,666,city_162,0.767,Male,1,no_enrollment,Masters,STEM,21.0,50-99,Funded Startup,4.0,8,0.0


We can now include the preformater in the pipeline.

In [20]:
full_pipeline = Pipeline(steps=[
    ('preformat', preformater),
    ('col_trans', col_trans),
    ('model', classifier)
])
display(full_pipeline)

Pipeline(steps=[('preformat',
                 FunctionTransformer(func=<function preformat at 0x7be1a62f7560>)),
                ('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

We must then redo the train/test splitting, starting from the non-preformatted dataframe.

In [21]:
X = data[num_cols+cat_cols]
y = data['target']
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)

In [22]:
full_pipeline.fit(X_train, y_train)
y_pred2 = full_pipeline.predict(X_test)
score2 = full_pipeline.score(X_test, y_test)
print('\nModel score:', score2)


Model score: 0.7713987473903967


## 6. Hyperparameters tuning

In [23]:
# TBD text

Get the list of tuneable parameters:\
For each pipeline step, we get a summary, then the list of parameters.\
We have multiple, parallel stages, so a lot of tuneable parameters.\
The parameters corresponding to a given step are names as follows:

    step1__step2__step3__ ... parameter: value

for instance:

    'col_trans__cat_pipeline__one-hot__handle_unknown': 'ignore'

In [24]:
full_pipeline.get_params()

{'memory': None,
 'steps': [('preformat',
   FunctionTransformer(func=<function preformat at 0x7be1a62f7560>)),
  ('col_trans',
   ColumnTransformer(n_jobs=-1,
                     transformers=[('num_pipeline',
                                    Pipeline(steps=[('impute', SimpleImputer()),
                                                    ('scale', MinMaxScaler())]),
                                    ['city_development_index',
                                     'relevent_experience', 'experience',
                                     'last_new_job', 'training_hours']),
                                   ('cat_pipeline',
                                    Pipeline(steps=[('impute',
                                                     SimpleImputer(strategy='most_frequent')),
                                                    ('one-hot',
                                                     OneHotEncoder(handle_unknown='ignore',
                                                  

### 6.1 Pipeline hyperparameters tuning

1. create a dictionnary of hyperparameters to tune:

In [25]:
hyper_params = {'model__penalty' : [None, 'l2'],
                'model__C' : np.logspace(-2, 2, 10)}

2. Run the pipeline with an hyperparameters tuning algorithm such as GridSearch:\
Note: This is possible because our pipeline contains a 'model' stage.

In [26]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(full_pipeline, hyper_params, cv=5, scoring='accuracy')

import warnings
with warnings.catch_warnings():
    # disables the 'UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters' warning
    warnings.filterwarnings("ignore", message="Setting penalty=None will ignore the C and l1_ratio parameters")
    gs.fit(X_train, y_train)

3. Inspect the outcome of GridSearch

In [27]:
print("Best Score of train set: "+str(gs.best_score_))
print("Best parameter set: "+str(gs.best_params_))
print("Test Score: "+str(gs.score(X_test,y_test)))

Best Score of train set: 0.7661492621809053
Best parameter set: {'model__C': 1.6681005372000592, 'model__penalty': 'l2'}
Test Score: 0.7716597077244259


The best set of hyperparameters (among those included in GridSearch) requires a 'l2' regularization and a value of 1.67 for the C parameter of the model.

In practice, the selection of the best set of parameters should include both the 'scoring' value and the 'mean_score_time':\
if the computing time is multiplied by 10, with a marginal 'scoring' improvement, the best model is (likely, it ultimately depends on the use case) the fastest one.

### 6.2 Using multiple scoring methods

It is possible to have GridSearch work with a list of scorers, instead of a unique scoring keyword.\
However, GridSearch will only optimize the scorer present in 'refit' (with a unique scorer, refit is True by default).\
The other scorer present in the list will simply be monitored.

In [50]:
scores = ['f1', 'accuracy']

It is possible to define a custom scorer that will be optimized against:

In [72]:
def custom_scorer(cv_results_):

    score = np.argmax(cv_results_['mean_test_accuracy'] + cv_results_['mean_test_f1'])
    return score

In [75]:
gs_mult = GridSearchCV(full_pipeline, hyper_params, cv=5, scoring=custom_scorer(gs_mult.cv_results_))

import warnings
with warnings.catch_warnings():
    # disables the 'UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters' warning
    warnings.filterwarnings("ignore", message="Setting penalty=None will ignore the C and l1_ratio parameters")
    gs_mult.fit(X_train, y_train)

KeyError: 'mean_test_accuracy'

In [71]:
gs_mult.cv_results_.keys()

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_model__C', 'param_model__penalty', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

In [ ]:
print("Best Score of train set: "+str(gs.best_score_))
print("Best parameter set: "+str(gs.best_params_))
print("Test Score: "+str(gs.score(X_test,y_test)))

### 6.3 Selecting the best method

Previously we selected a method and tuned the relevant hyperparameters for this method.\
Now we want to select the best method.\
To achieve this, the idea is to implement several methods and switch them on alternatively within GridSearch, in order to find which combination reaches the best score.\
As an example, we will investigate which scaler performs best with the current dataset, StandardScaler or MinMaxScaler.

1. Create a new version of the pipeline branch for numerical columns:

In [35]:
from sklearn.preprocessing import StandardScaler

num_pipeline2 = Pipeline(steps=[
    ('impute', SimpleImputer(strategy='mean')),
    ('minmax_scale', MinMaxScaler()),
    ('std_scale', StandardScaler()),
])

This branch now contains two flavours for a scaler, StandardScaler or MinMaxScaler.\
Note: We do not want to run them successively, but alternatively.

2. Since num_pipeline was itself a branch of col_trans, we need to reflect the change in col_trans as well:\
num_pipeline is replaced by num_pipeline2, cat_pipeline remains unchanged. 

In [36]:
col_trans2 = ColumnTransformer(transformers=[
    ('num_pipeline',num_pipeline2,num_cols),
    ('cat_pipeline',cat_pipeline,cat_cols)
    ],
    remainder='drop',
    n_jobs=-1)

3. Since col_trans was a step within full_pipeline, full_pipeline is modified accordingly.

In [37]:
new_pipeline = Pipeline(steps=[
    ('preformat', preformater),
    ('col_trans', col_trans2),
    ('model', classifier)
])
display(new_pipeline)

Pipeline(steps=[('preformat',
                 FunctionTransformer(func=<function preformat at 0x7be1a62f7560>)),
                ('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('minmax_scale',
                                                                   MinMaxScaler()),
                                                                  ('std_scale',
                                                                   StandardScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

4. Create a list of dictionaries (instead of a single dictionnary) for the grid search parameters.\
   In the GridSearch parameters, the steps we want to **skip** must be explicitly specified, and their value set to 'passthrough'.\
   Nothing changes for the other parameters within GridSearch.

In [38]:
scaler_params = [{'col_trans__num_pipeline__minmax_scale': ['passthrough'],
                     'model__penalty' : [None, 'l2'],
                     'model__C' : np.logspace(-2, 2, 10)},
                    {'col_trans__num_pipeline__std_scale': ['passthrough'],
                     'model__penalty' : [None, 'l2'],
                     'model__C' : np.logspace(-2, 2, 10)}]

When running GridSearch with the first dictionnary of parameters, we skip the MinMaxScaler.\
When running GridSearch with the second dictionnary of parameters, we skip the StandardScaler.\.

5. Run the pipeline with GridSearch.

In [39]:
gs2 = GridSearchCV(new_pipeline, scaler_params, cv=5, scoring='accuracy')

with warnings.catch_warnings():
    # disables the 'UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters' warning
    warnings.filterwarnings("ignore", message="Setting penalty=None will ignore the C and l1_ratio parameters")
    gs2.fit(X_train, y_train)

6. Inspect the outcome of GridSearch.

In [42]:
print("Best Score of train set: "+str(gs2.best_score_))
print("Best parameter set: "+str(gs2.best_params_))
print("Test Score: "+str(gs2.score(X_test,y_test)))

Best Score of train set: 0.7668016204671773
Best parameter set: {'col_trans__num_pipeline__minmax_scale': 'passthrough', 'model__C': 0.01, 'model__penalty': 'l2'}
Test Score: 0.7693110647181628


The best score is achieved with 'col_trans__num_pipeline__minmax_scale': 'passthrough', so the best method is the StandardScaler.\
For the other parameters, the best values are 'l2' for the 'penalty' hyperparameter and 0.01 for the 'C' hyperparameter.

7. Display the entire dataframe of GridSearch results.

In [44]:
pd.DataFrame(gs2.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_col_trans__num_pipeline__minmax_scale,param_model__C,param_model__penalty,param_col_trans__num_pipeline__std_scale,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.451802,0.316023,0.267161,0.270692,passthrough,0.01,None,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.761252,0.770962,0.767047,0.766069,0.765090,0.766084,0.003132,9
1,0.195900,0.035249,0.044076,0.003782,passthrough,0.01,l2,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.765166,0.768352,0.768679,0.767047,0.764763,0.766802,0.001601,1
2,0.212943,0.039282,0.047591,0.003341,passthrough,0.027826,None,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.761252,0.770962,0.767047,0.766069,0.765090,0.766084,0.003132,9
3,0.201926,0.056549,0.046080,0.001743,passthrough,0.027826,l2,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.764188,0.766721,0.769331,0.766069,0.763458,0.765953,0.002067,22
4,0.185698,0.003449,0.050400,0.001687,passthrough,0.077426,None,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.761252,0.770962,0.767047,0.766069,0.765090,0.766084,0.003132,9
5,0.229500,0.055284,0.045238,0.002228,passthrough,0.077426,l2,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.761905,0.768679,0.768026,0.766721,0.764437,0.765954,0.002490,21
6,0.208877,0.035529,0.046080,0.015546,passthrough,0.215443,None,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.761252,0.770962,0.767047,0.766069,0.765090,0.766084,0.003132,9
7,0.236680,0.017070,0.049779,0.003890,passthrough,0.215443,l2,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.762231,0.770310,0.768026,0.766395,0.764763,0.766345,0.002756,2
8,0.236564,0.074376,0.047009,0.006421,passthrough,0.599484,None,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.761252,0.770962,0.767047,0.766069,0.765090,0.766084,0.003132,9
9,0.193657,0.020329,0.047627,0.007970,passthrough,0.599484,l2,NaN,{'col_trans__num_pipeline__minmax_scale': 'pas...,0.761905,0.770310,0.767700,0.766069,0.765416,0.766280,0.002764,3


### 6.3 Selecting the best model.

In [ ]:
tbd

To evaluate several models, it is possible to:
1. Create several pipelines, run them successively with Gridsearch, collect the results in a single dataframe, and select the best suited model.
2. Create a switcher class that works for any estimator. We will not investigate this path in this course.

## 7 Saving a pipeline

### 7.1 with pickle

It is possible to save a scikit-learn model/pipeline using Python’s pickle

In [49]:
import pickle

with open('save_pickle.pkl', 'wb') as f:
    pickle.dump(full_pipeline, f)

with open('save_pickle.pkl', 'rb') as f:
    load_pickle = pickle.load(f)

display(load_pickle)

Pipeline(steps=[('preformat',
                 FunctionTransformer(func=<function preformat at 0x7be1a62f7560>)),
                ('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

### 7.2 with joblib

joblib’s replacement of pickle (dump, load) is more efficient on objects containing large numpy arrays.\
This is often the case for (fitted) scikit-learn models/pipelines.

In [47]:
from joblib import dump, load
dump(full_pipeline, 'save_full_pipeline.joblib') 
load_joblib = load('save_full_pipeline.joblib') 
display(load_joblib)

Pipeline(steps=[('preformat',
                 FunctionTransformer(func=<function preformat at 0x7be1a62f7560>)),
                ('col_trans',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('num_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer()),
                                                                  ('scale',
                                                                   MinMaxScaler())]),
                                                  ['city_development_index',
                                                   'relevent_experience',
                                                   'experience', 'last_new_job',
                                                   'training_hours']),
                                                 ('cat_pipeline',
                                                  Pipeline(steps=[('impute',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('one-hot',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['gender',
                                                   'enrolled_university',
                                                   'education_level',
                                                   'major_discipline',
                                                   'company_size',
                                                   'company_type'])])),
                ('model', LogisticRegression(random_state=0))])

Note: Never load pickle/joblib objects from an untrusted source, they might contain malicious executable code.

In order to rebuild a similar model with future versions of scikit-learn, additional metadata should be saved along the model/pipeline:
- The training data, e.g. a reference to an immutable snapshot
- The python source code used to generate the model
- The versions of scikit-learn and its dependencies
- The cross validation score obtained on the training data